In [1]:
import torch
import pickle
import pandas as pd
import numpy as np
from src.train import *
from src.processing import *
from src.models import *
from src.inference import *
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertModel
from transformers import DistilBertModel, DistilBertTokenizer
from torch.utils.data import Dataset, DataLoader

torch.manual_seed(42)
np.random.seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

torch.cuda.empty_cache()

Using device: cuda


In [3]:
vocab = Vocabulary()
tokenizer = BertTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
model = BertModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')

ratings_df, movie_descriptions, movies_metadata = create_ratings_df(
    number_of_movies=7500,
    links_path='CLIP4Rec/archive/links.csv',
    movies_metadata_path='CLIP4Rec/archive/movies_metadata.csv',
    ratings_path='CLIP4Rec/archive/ratings.csv'
    )
sequences = get_sequences(ratings_df)
vocab.build_vocab(sequences)

train_sentences, val_sentences = train_test_split(sequences, test_size=0.2, random_state=42)
train_data, film_descriptions_encoded = prepare_dataset(
    train_sentences, movie_descriptions, tokenizer, vocab, encode_descriptions=True
)
val_data = prepare_dataset(
    val_sentences, movie_descriptions, tokenizer, vocab
)

train_dataset = FilmRecommendationDataset(train_data, film_descriptions_encoded)
val_dataset = FilmRecommendationDataset(val_data, film_descriptions_encoded)

epochs = 7
batch_size = 32
lr = 0.0001

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

/teamspace/studios/this_studio/CLIP4Rec/src/processing.py:20: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_metadata = pd.read_csv(movies_metadata_path)
100%|██████████| 216271/216271 [00:00<00:00, 423374.40it/s]


In [8]:
film_encoder = SASFilmEncoder(item_num=len(vocab.word_to_index), seq_len=seq_len, embed_dim=384, device=device)
text_encoder = TextEncoder(model, output_dim=384, add_fc_layer=False)

In [7]:
train_clip(film_encoder, text_encoder, train_loader, val_loader, 
           epochs=epochs, lr=lr, device=device, iter_verbose=1000, folder='CLIP4Rec/artifacts')

  7%|▋         | 1000/13517 [05:29<1:08:50,  3.03it/s]

Epoch 1, Batch 1000
Accuracy: 0.0077
Agreggated loss: 2.9500
Classification loss: 7.3598
Contrastive loss: 1.2889


 15%|█▍        | 2000/13517 [10:59<1:02:44,  3.06it/s]

Epoch 1, Batch 2000
Accuracy: 0.0161
Agreggated loss: 2.9035
Classification loss: 7.1817
Contrastive loss: 1.2326


 22%|██▏       | 3000/13517 [16:28<57:43,  3.04it/s]  

...

Epoch 5, Batch 13000
Accuracy: 0.0562
Agreggated loss: 2.6254
Classification loss: 5.6919
Contrastive loss: 1.0651


100%|██████████| 13517/13517 [1:14:13<00:00,  3.04it/s]


Epoch 5, Batch 13517
Accuracy: 0.0556
Agreggated loss: 2.6255
Classification loss: 5.6933
Contrastive loss: 1.0649


100%|██████████| 3380/3380 [14:58<00:00,  3.76it/s]


Epoch 5: Val Loss: 2.6517, Val Accuracy: 0.0452
Val Classification loss: 6.1001
Val Contrastive loss: 0.9987


In [8]:
# torch.save(film_encoder.state_dict(), 'artifacts/film_encoder_weights_final.pth')
# torch.save(text_encoder.state_dict(), 'artifacts/text_encoder_weights_final.pth')

torch.save(train_dataset, 'CLIP4Rec/artifacts/train_dataset.pt')
torch.save(val_dataset, 'CLIP4Rec/artifacts/val_dataset.pt')

with open('CLIP4Rec/artifacts/ratings_df.pickle', 'wb') as f:
  pickle.dump(ratings_df, f)

with open('CLIP4Rec/artifacts/movie_descriptions.pickle', 'wb') as f:
  pickle.dump(movie_descriptions, f)

with open('CLIP4Rec/artifacts/sequences.pickle', 'wb') as f:
  pickle.dump(sequences, f)

with open('CLIP4Rec/artifacts/vocab.pickle', 'wb') as f:
  pickle.dump(vocab, f)

with open('CLIP4Rec/artifacts/film_descriptions_encoded.pickle', 'wb') as f:
  pickle.dump(film_descriptions_encoded, f)

with open('CLIP4Rec/artifacts/movies_metadata.pickle', 'wb') as f:
  pickle.dump(movies_metadata, f)

In [2]:
# list_movies = ["Only Lovers Left Alive",
#                "The Twilight Saga: Eclipse",
#                "Me Before You",
#                "(500) Days of Summer"]

list_movies = ["Minions",
               "Zootopia",
               "Shrek",
               "Kung Fu Panda"]

In [3]:
vocab = pd.read_pickle('artifacts/vocab.pickle')
movies_metadata = pd.read_pickle('artifacts/movies_metadata.pickle')
film_descriptions_encoded = pd.read_pickle('artifacts/film_descriptions_encoded.pickle')
bert_model = DistilBertModel.from_pretrained('distilbert-base-uncased')
bert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

dim = 384
num_trees=10
search_type='euclidean'

/Users/Aleksandra.Sorokovikova/Study/Data Science/mamba rec system/CLIP4Rec/venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

In [4]:
# build and save

inference = Inference(
    film_encoder_path = 'artifacts/film_encoder_weights_final_4.pth',
    text_encoder_path = 'artifacts/text_encoder_weights_final_4.pth',
    vocab=vocab,
    dim=dim,
    movies_metadata=movies_metadata,
    seq_len=seq_len,
    device=device,
    bert_model=bert_model,
    bert_tokenizer=bert_tokenizer,
)

film_embeddings, text_embeddings = inference.get_embeddings(film_descriptions_encoded, batch_size=32)

annoy_model = AnnoySearchEngine(
    dim=dim,
    num_trees=num_trees,
    search_type=search_type,
)
annoy_model.build_trees(film_embeddings, text_embeddings)
annoy_model.save_indexes('artifacts/text_index.ann', 'artifacts/film_index.ann', 'artifacts/idx_to_movieId.pickle')
inference.init_annoy_model('artifacts/text_index.ann', 'artifacts/film_index.ann', 'artifacts/idx_to_movieId.pickle', num_trees=10)

100%|██████████| 7314/7314 [00:01<00:00, 7024.77it/s]


In [5]:
overview = movies_metadata.query('title=="Kung Fu Panda"')['overview'].values[0]
inference.search_text(overview, in_films=True)

['Kung Fu Panda',
 'Kung Fu Panda 2',
 'The Mermaid',
 'Kung Fu Dunk',
 'The Man with the Iron Fists 2',
 'Girls Against Boys',
 'Shanghai Knights',
 'Clean',
 'Rise: Blood Hunter',
 'Saving Mr. Wu']

In [11]:
# load and init

inference = Inference(
    film_encoder_path = 'artifacts/film_encoder_weights_final_4.pth',
    text_encoder_path = 'artifacts/text_encoder_weights_final_4.pth',
    vocab=vocab,
    dim=384,
    movies_metadata=movies_metadata,
    seq_len=seq_len,
    device=device,
    bert_model=bert_model,
    bert_tokenizer=bert_tokenizer,
)

inference.init_annoy_model('artifacts/text_index.ann', 'artifacts/film_index.ann', 'artifacts/idx_to_movieId.pickle', num_trees=10)

In [8]:
overview = movies_metadata.query('title=="Megamind"')['overview'].values[0]
inference.search_text(overview, in_films=True)

['Megamind',
 'Despicable Me 2',
 'Teenage Mutant Ninja Turtles: Out of the Shadows',
 'The Lego Movie',
 'Teen Titans: Trouble in Tokyo',
 'LEGO DC Comics Super Heroes: Justice League: Attack of the Legion of Doom!',
 'Superman/Shazam!: The Return of Black Adam',
 'The SpongeBob SquarePants Movie',
 'In the Name of the King 2: Two Worlds',
 'Fantastic 4: Rise of the Silver Surfer']